In [1]:
# First Import the Pandas and Numpy Packages

import pandas as pd
import numpy as np
import pathlib

In [2]:
# Set Up Export and Export Paths
root_path = r'C:\Users\MLeif\data_science_practice'
project_folder = '1_descriptive_stats'
data_folder = 'datasets'

import_path = pathlib.Path(root_path).joinpath(project_folder).joinpath(data_folder)
export_path = pathlib.Path(root_path).joinpath(project_folder).joinpath(data_folder)

print(import_path)
print(export_path)

C:\Users\MLeif\data_science_practice\1_descriptive_stats\datasets
C:\Users\MLeif\data_science_practice\1_descriptive_stats\datasets


In [3]:
#Import the imdb dataset, creaate an index column called movie ID, and check the first five rows
import_data_file = 'imdb.csv'
import_file = import_path.joinpath(import_data_file)

imdb_source = pd.read_csv(import_file).drop_duplicates()
imdb_source['movie_id'] = imdb_source.index
imdb_source = imdb_source.set_index('movie_id')
imdb_source.head()

,Name,Date,Rate,Votes,Genre,Duration,Type,Certificate,Episodes,Nudity,Violence,Profanity,Alcohol,Frightening
movie_id,,,,,,,,,,,,,,
0,No Time to Die,2021,7.6,"107,163","Action, Adventure, Thriller",163,Film,PG-13,-,Mild,Moderate,Mild,Mild,Moderate
1,The Guilty,2021,6.3,"64,375","Crime, Drama, Thriller",90,Film,R,-,None,None,Severe,None,Moderate
2,The Many Saints of Newark,2021,6.4,"27,145","Crime, Drama",120,Film,R,-,Moderate,Severe,Severe,Moderate,Moderate
3,Venom: Let There Be Carnage,2021,6.4,"30,443","Action, Adventure, Sci-Fi",97,Film,PG-13,-,None,Moderate,Moderate,Mild,Moderate
4,Dune,2021,8.3,"84,636","Action, Adventure, Drama",155,Film,PG-13,-,None,Moderate,None,Mild,Moderate


In [4]:
# Look at the number of Rows and Columns in the Data, to help verify the data later

imdb_source.shape

(5028, 14)

In [5]:
# Check that all of the Data Types Imported Correctly

imdb_source.dtypes

#Duration, Votes and Rate do Not Have the Correct Data Type, so that needs to be fixed

Name           object
Date            int64
Rate           object
Votes          object
Genre          object
Duration       object
Type           object
Certificate    object
Episodes       object
Nudity         object
Violence       object
Profanity      object
Alcohol        object
Frightening    object
dtype: object

In [6]:
# Thankfully it does not appear we have missing values directly in the source data

imdb_source.isnull().sum()

Name           0
Date           0
Rate           0
Votes          0
Genre          0
Duration       0
Type           0
Certificate    0
Episodes       0
Nudity         0
Violence       0
Profanity      0
Alcohol        0
Frightening    0
dtype: int64

In [7]:
# Filter Out any Films with Planned Releases by dropping 2021 onwards
print('Total Films Before 2020: ' + str(imdb_source[imdb_source['Date'] <= 2020]['Date'].count()))
print('Total Films After 2020: ' + str(imdb_source[imdb_source['Date'] > 2020]['Date'].count()))

imdb_source = imdb_source[imdb_source['Date'] <= 2020]

print('Total Films Now: ' + str(imdb_source['Date'].count()))

Total Films Before 2020: 4262
Total Films After 2020: 766
Total Films Now: 4262


In [8]:
#First Trim and Lowercase all of the string columns to standardize them, and rename the release year

imdb_wip = imdb_source

imdb_wip['film_name'] = imdb_wip['Name'].str.strip().str.lower()
imdb_wip['genre_list'] = imdb_wip['Genre'].str.strip().str.lower()
imdb_wip['medium_type'] = imdb_wip['Type'].str.strip().str.lower()
imdb_wip['age_rating'] = imdb_wip['Certificate'].str.strip().str.upper()
imdb_wip['release_year'] = imdb_wip['Date']


print(str(imdb_wip[['film_name','genre_list','medium_type','age_rating','release_year']].sample(10)))

                                film_name                  genre_list  \
movie_id                                                                
2049                         flash gordon   action, adventure, sci-fi   
3779                                  jfk    drama, history, thriller   
3819               underworld: blood wars  action, adventure, fantasy   
3909                        tra(sgre)dire               comedy, drama   
3379                      f is for family    animation, comedy, drama   
4475                           frances ha      comedy, drama, romance   
610                          white collar        comedy, crime, drama   
2750      how to sell drugs online (fast)        comedy, crime, drama   
808                             the chair               comedy, drama   
1740                 crazy, stupid, love.      comedy, drama, romance   

         medium_type age_rating  release_year  
movie_id                                       
2049            film       

In [9]:
#The votes column uses commas in the numbers which means they cannot be turned into numeric variables

imdb_wip['votes_fl'] = imdb_wip['Votes'].str.replace(',','')

print("Sample of Votes:\n" + str(imdb_wip[['Votes','votes_fl']].sample(5)))

Sample of Votes:
            Votes votes_fl
movie_id                  
3869        4,365     4365
2325       41,327    41327
1353      114,924   114924
1859      183,780   183780
4085      252,026   252026


In [10]:
#The Rate, Episodes and Duration columns use strings for missing values. 
#We need to change these to null so we can change the datatype

print("Total Rate Strings Before: " + str(imdb_wip[imdb_wip['Rate'] == 'No Rate']['Rate'].count()))
print("Total Duration Strings Before: " + str(imdb_wip[imdb_wip['Duration'] == 'None']['Duration'].count()))
print("Total Episodes Strings Before: " + str(imdb_wip[imdb_wip['Episodes'] == '-']['Episodes'].count()))
print("Total Votes Strings Before: " + str(imdb_wip[imdb_wip['votes_fl'] == 'No Votes']['votes_fl'].count()))

imdb_wip.loc[imdb_wip['Rate'] == 'No Rate', 'Rate'] = np.nan
imdb_wip.loc[imdb_wip['Duration'] == 'None', 'Duration'] = np.nan
imdb_wip.loc[imdb_wip['Episodes'] == '-', 'Episodes'] = np.nan
imdb_wip.loc[imdb_wip['votes_fl'] == 'No Votes', 'votes_fl'] = 0

print("Total Rate Nulls After: " + str( imdb_wip['Rate'].isnull().sum()))
print("Total Duration Nulls After: " + str( imdb_wip['Duration'].isnull().sum()))
print("Total Episodes Nulls After: " + str( imdb_wip['Episodes'].isnull().sum()))
print("Total vote Nulls After: " + str(imdb_wip[imdb_wip['votes_fl'] == 0]['votes_fl'].count()))

Total Rate Strings Before: 38
Total Duration Strings Before: 121
Total Episodes Strings Before: 3130
Total Votes Strings Before: 38
Total Rate Nulls After: 38
Total Duration Nulls After: 121
Total Episodes Nulls After: 3130
Total vote Nulls After: 38


In [11]:
# Set the Columns for Votes, Rate, Duration, and Episodes to floats

imdb_wip['avg_score'] = imdb_wip['Rate'].astype('float64')
imdb_wip['total_votes'] = imdb_wip['votes_fl'].astype('float64')
imdb_wip['film_duration'] = imdb_wip['Duration'].astype('float64')
imdb_wip['total_episodes'] = imdb_wip['Episodes'].astype('float64')

print(imdb_wip[['avg_score','total_votes','film_duration','total_episodes']].dtypes)
print(imdb_wip[['avg_score','total_votes','film_duration','total_episodes']].sample(5))


avg_score         float64
total_votes       float64
film_duration     float64
total_episodes    float64
dtype: object
          avg_score  total_votes  film_duration  total_episodes
movie_id                                                       
4304            6.7       3906.0           44.0            58.0
2329            6.9      66789.0          131.0             NaN
1504            7.9     482833.0          103.0             NaN
4260            6.4      52927.0          105.0             NaN
3699            8.1      99578.0          145.0             NaN


In [12]:
# Create a List of Certifications which will be changed into Null Later

null_cert_list = ['(Banned)','Approved','E','None','Not Rated','Passed','Unrated']

In [13]:
# Recode some Age Ratings Which Represent the same Ratings

print("List Certificate Categories Before: \n" + str(imdb_source['age_rating'].unique()))

imdb_wip.loc[imdb_wip['Certificate'].isin(null_cert_list), 'age_rating'] = np.nan
imdb_wip.loc[imdb_wip['Certificate'].isin(['X']), 'age_rating'] = 'NC-17'
imdb_wip.loc[imdb_wip['Certificate'].isin(['M/PG','M','GP']), 'age_rating'] = 'PG'
imdb_wip.loc[imdb_wip['Certificate'].isin(['TV-Y7-FV']), 'age_rating'] = 'TV-Y7'

print("List Age Rating Categories After: \n" + str(imdb_source['age_rating'].unique()))

List Certificate Categories Before: 
['PG-13' 'TV-14' 'TV-MA' 'R' 'PG' 'TV-PG' 'TV-G' 'NONE' '(BANNED)'
 'NOT RATED' 'NC-17' 'TV-Y7-FV' 'UNRATED' 'APPROVED' 'G' 'TV-Y' 'TV-Y7'
 'GP' 'PASSED' 'E' 'M' 'X' 'M/PG']
List Age Rating Categories After: 
['PG-13' 'TV-14' 'TV-MA' 'R' 'PG' 'TV-PG' 'TV-G' nan 'NC-17' 'TV-Y7' 'G'
 'TV-Y']


In [14]:
# Here we are adding a new field to change define wether each rating is for TV or Film

def calc_certificate_type(cert):
    
    if cert in null_cert_list:
        return np.nan
    elif 'TV' in cert:
        return 'tv'
    else:
        return 'film'
    
imdb_wip['age_rating_type'] = imdb_wip['Certificate'].apply(lambda x: calc_certificate_type(x))

print(imdb_wip[['age_rating_type','age_rating']].drop_duplicates())

         age_rating_type age_rating
movie_id                           
25                  film      PG-13
27                    tv      TV-14
28                    tv      TV-MA
35                  film          R
59                  film         PG
105                   tv      TV-PG
189                   tv       TV-G
198                  NaN        NaN
471                 film      NC-17
500                   tv      TV-Y7
889                 film          G
896                   tv       TV-Y


In [15]:
# Define what level of certification the rating is, in order to make the variable Ordinal

def calc_certificate_ordinal(cert_cat):
    
    if (cert_cat == 'G') | (cert_cat == 'TV-Y'):
        return 1
    elif (cert_cat == 'PG') | (cert_cat == 'TV-Y7'):
        return 2
    elif (cert_cat == 'PG-13') | (cert_cat == 'TV-G'):
        return 3
    elif (cert_cat == 'R') | (cert_cat == 'TV-PG'):
        return 4
    elif (cert_cat == 'NC-17') | (cert_cat == 'TV-14'):
        return 5
    elif (cert_cat == 'TV-MA'):
        return 6
    else:
        return np.nan
    
imdb_wip['age_rating_num'] = imdb_wip['age_rating'].apply(lambda x: calc_certificate_ordinal(x))

print(imdb_wip[['age_rating_num','age_rating']].drop_duplicates())
    
#NOTE: Each of these Levels is not designed to be Equivelent between TV and Film, 
# this and the next UDF are the most efficient way to make the age ratings ordinal

          age_rating_num age_rating
movie_id                           
25                   3.0      PG-13
27                   5.0      TV-14
28                   6.0      TV-MA
35                   4.0          R
59                   2.0         PG
105                  4.0      TV-PG
189                  3.0       TV-G
198                  NaN        NaN
471                  5.0      NC-17
500                  2.0      TV-Y7
889                  1.0          G
896                  1.0       TV-Y


In [16]:
# Recode certification sub-categories to be ordinal

def recode_age_rating_category(cert_cat):
    
    if cert_cat == 'None':
        return 0
    elif cert_cat == 'Mild':
        return 1
    elif cert_cat == 'Moderate':
        return 2
    elif cert_cat == 'Severe':
        return 3
    else:
        return np.nan
    
imdb_wip['nudity_level'] = imdb_wip['Nudity'].apply(lambda x: recode_age_rating_category(x))
imdb_wip['violence_level'] = imdb_wip['Violence'].apply(lambda x: recode_age_rating_category(x))
imdb_wip['profanity_level'] = imdb_wip['Profanity'].apply(lambda x: recode_age_rating_category(x))
imdb_wip['alcohol_level'] = imdb_wip['Alcohol'].apply(lambda x: recode_age_rating_category(x))
imdb_wip['frightening_level'] = imdb_wip['Frightening'].apply(lambda x: recode_age_rating_category(x))

print('Nudity Level Counts: ' + str(imdb_wip['nudity_level'].value_counts()))
print('Violence Level Counts: ' + str(imdb_wip['violence_level'].value_counts()))
print('Profanity Level Counts: ' + str(imdb_wip['profanity_level'].value_counts()))
print('Alcohol Level Counts: ' + str(imdb_wip['alcohol_level'].value_counts()))
print('Frightening Level Counts: ' + str(imdb_wip['frightening_level'].value_counts()))

Nudity Level Counts: 1.0    1694
0.0    1059
2.0     901
3.0     344
Name: nudity_level, dtype: int64
Violence Level Counts: 2.0    1349
1.0    1241
3.0     930
0.0     450
Name: violence_level, dtype: int64
Profanity Level Counts: 1.0    1530
2.0    1209
3.0     758
0.0     477
Name: profanity_level, dtype: int64
Alcohol Level Counts: 1.0    2432
2.0     760
0.0     549
3.0     198
Name: alcohol_level, dtype: int64
Frightening Level Counts: 2.0    1481
1.0    1139
3.0     693
0.0     592
Name: frightening_level, dtype: int64


In [17]:
#Trim and Lowercase all type and genre columns to standardize them

imdb_source['genre_list'] = imdb_wip['genre_list'].str.split(',')
imdb_genre = imdb_wip.explode('genre_list')
imdb_genre['genre_list'] = imdb_genre['genre_list'].str.strip()
imdb_genre = pd.get_dummies(imdb_genre['genre_list'], prefix=('genre'))
imdb_genre_join = imdb_wip.join(imdb_genre.groupby(imdb_genre.index.get_level_values(0)).max())


print('Sample of Action Movies: \n' + str(imdb_genre_join[imdb_genre_join['genre_action'] == 1].sample(5)[['genre_list','genre_action']]))
print('Sample of Drama Movies: \n' + str(imdb_genre_join[imdb_genre_join['genre_drama'] == 1].sample(5)[['genre_list','genre_drama']]))
print('Sample of Comedy Movies: \n' + str(imdb_genre_join[imdb_genre_join['genre_comedy'] == 1].sample(5)[['genre_list','genre_comedy']]))

Sample of Action Movies: 
                             genre_list  genre_action
movie_id                                             
1812      [action,  mystery,  thriller]             1
2724      [action,  adventure,  comedy]             1
1171             [action,  drama,  war]             1
2486                  [action,  comedy]             1
2698       [action,  biography,  drama]             1
Sample of Drama Movies: 
                          genre_list  genre_drama
movie_id                                         
1122      [crime,  drama,  thriller]            1
1837       [crime,  drama,  mystery]            1
2536       [comedy,  drama,  family]            1
2721               [drama,  western]            1
2112        [crime,  drama,  horror]            1
Sample of Comedy Movies: 
                                genre_list  genre_comedy
movie_id                                                
3024                      [comedy,  drama]             1
2136      [animation,  a

In [18]:
# Create a Dataset of all Values Related to Television Serries

imbd_tv = imdb_genre_join[imdb_genre_join['medium_type'] == 'series']\
                 [['film_name', 'release_year','avg_score','total_votes','total_episodes','genre_list',\
                 'age_rating', 'age_rating_type', 'age_rating_num',\
                 'nudity_level','violence_level','profanity_level','alcohol_level','frightening_level',\
                 'genre_action','genre_adventure','genre_animation','genre_biography','genre_comedy','genre_crime',\
                 'genre_documentary','genre_drama','genre_family','genre_fantasy','genre_film-noir','genre_game-show',\
                 'genre_history','genre_horror','genre_music','genre_musical','genre_mystery','genre_news',\
                 'genre_reality-tv','genre_romance','genre_sci-fi','genre_short','genre_sport','genre_talk-show',\
                 'genre_thriller', 'genre_war', 'genre_western']]

In [19]:
# Create a Dataset of all Values Related to Movies

imbd_film = imdb_genre_join[imdb_genre_join['medium_type'] == 'film']\
                 [['film_name', 'release_year','avg_score','total_votes','film_duration','genre_list',\
                 'age_rating', 'age_rating_num',\
                 'nudity_level','violence_level','profanity_level','alcohol_level','frightening_level',\
                 'genre_action','genre_adventure','genre_animation','genre_biography','genre_comedy','genre_crime',\
                 'genre_documentary','genre_drama','genre_family','genre_fantasy','genre_film-noir',\
                 'genre_history','genre_horror','genre_music','genre_musical','genre_mystery','genre_news',\
                 'genre_romance','genre_sci-fi','genre_short','genre_sport',\
                 'genre_thriller', 'genre_war', 'genre_western']]

In [21]:
imdb_genre_join.describe()

,Date,release_year,avg_score,total_votes,film_duration,total_episodes,age_rating_num,nudity_level,violence_level,profanity_level,...,genre_news,genre_reality-tv,genre_romance,genre_sci-fi,genre_short,genre_sport,genre_talk-show,genre_thriller,genre_war,genre_western
count,4262.000000,4262.000000,4224.000000,4.262000e+03,4141.000000,1132.000000,3872.000000,3998.000000,3970.000000,3974.000000,...,4262.000000,4262.000000,4262.000000,4262.000000,4262.000000,4262.000000,4262.000000,4262.000000,4262.000000,4262.000000
mean,2003.463163,2003.463163,6.955942,1.483937e+05,96.181840,132.732332,3.789514,1.132566,1.694962,1.565677,...,0.000235,0.007039,0.122008,0.102299,0.004458,0.013139,0.000469,0.160488,0.011966,0.010793
std,15.695409,15.695409,1.005544,2.138061e+05,38.991316,680.042699,1.152251,0.903907,0.952592,0.931367,...,0.015318,0.083612,0.327334,0.303077,0.066627,0.113885,0.021660,0.367101,0.108746,0.103340
min,1922.000000,1922.000000,1.900000,0.000000e+00,2.000000,3.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1996.000000,1996.000000,6.400000,2.261150e+04,81.000000,17.000000,3.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2008.000000,2008.000000,7.100000,7.641700e+04,101.000000,35.500000,4.000000,1.000000,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2016.000000,2016.000000,7.700000,1.861498e+05,118.000000,97.000000,4.000000,2.000000,2.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2020.000000,2020.000000,9.700000,2.474122e+06,582.000000,10336.000000,6.000000,3.000000,3.000000,3.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
imdb_genre_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4262 entries, 25 to 5027
Data columns (total 58 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               4262 non-null   object 
 1   Date               4262 non-null   int64  
 2   Rate               4224 non-null   object 
 3   Votes              4262 non-null   object 
 4   Genre              4262 non-null   object 
 5   Duration           4141 non-null   object 
 6   Type               4262 non-null   object 
 7   Certificate        4262 non-null   object 
 8   Episodes           1132 non-null   object 
 9   Nudity             4262 non-null   object 
 10  Violence           4262 non-null   object 
 11  Profanity          4262 non-null   object 
 12  Alcohol            4262 non-null   object 
 13  Frightening        4262 non-null   object 
 14  film_name          4262 non-null   object 
 15  genre_list         4262 non-null   object 
 16  medium_type        4262

In [26]:
#Export a file with all of the Columns in case we need it later
export_data_file = imdb_genre_join
export_data_name = 'imdb_clean.csv'
export_file = export_path.joinpath(export_data_name)

export_data_file.to_csv(export_file, index=True)

In [27]:
#Export a file with all the TV Serries Data

export_data_file = imbd_tv
export_data_name = 'imdb_tv_clean.csv'
export_file = export_path.joinpath(export_data_name)

export_data_file.to_csv(export_file, index=True)

In [28]:
#Export a file with all the TV Serries Data

export_data_file = imbd_film
export_data_name = 'imdb_film_clean.csv'
export_file = export_path.joinpath(export_data_name)

export_data_file.to_csv(export_file, index=True)